In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pingouin as pg
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

### Data loading

In [3]:
origin = pd.read_csv("/Users/rmgaliullin/cam_data_600k/data/all.csv")

/var/folders/_7/w61_y6mj5bgdr26d9sk7tmps8zhr5b/T/ipykernel_6549/2819353598.py:1: DtypeWarning: Columns (15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  origin = pd.read_csv("/Users/rmgaliullin/cam_data_600k/data/all.csv")


In [4]:
df = origin[["repo", "java_file"]]
columns = ['cc', 'nobl', 'loc', 'hse', 'hsd', 'hsv', 'coco', 'midx', 'nocl']
# Perform inner join on 'repo' and 'java_file'
merged_df = df.copy()
for i in columns:
    temp = pd.read_csv(f"/Users/rmgaliullin/cam_data_600k/data/{i}.csv")
    temp = temp.drop_duplicates(['repo', 'java_file'])
    merged_df = pd.merge(merged_df, temp, on=['repo', 'java_file'], how='inner')

In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606000 entries, 0 to 605999
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   repo       606000 non-null  object 
 1   java_file  606000 non-null  object 
 2   cc         606000 non-null  int64  
 3   nobl       606000 non-null  int64  
 4   loc        606000 non-null  int64  
 5   hse        606000 non-null  float64
 6   hsd        606000 non-null  float64
 7   hsv        606000 non-null  float64
 8   coco       606000 non-null  int64  
 9   midx       606000 non-null  float64
 10  nocl       606000 non-null  int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 50.9+ MB


#### Check that all.csv == concatenation of loc.csv, cc.csv etc

In [6]:
test = pd.merge(origin, merged_df, on=['repo', 'java_file'], suffixes=('_df1', '_df2'), how='right')
test= test[test['repo'] != 'yegor256/jaxec']

assert (test['cc_df1'].astype(int) == test['cc_df2']).all()
assert (test['coco_df1'].astype(int) == test['coco_df2']).all()
assert (test['hsv_df1'].astype(float) == test['hsv_df2']).all()
assert (test['hse_df1'].astype(float) == test['hse_df2']).all()
assert (test['hsd_df1'].astype(float) == test['hsd_df2']).all()
assert (test['loc_df1'].astype(int) == test['loc_df2']).all()
assert (test['nobl_df1'].astype(int) == test['nobl_df2']).all()
assert (test['midx_df1'].astype(float) == test['midx_df2']).all()
assert (test['nocl_df1'].astype(int) == test['nocl_df2']).all()
# pd.merge(origin[origin['loc']=='-'], merged_df, on=['repo', 'java_file'], suffixes=('_df1', '_df2'), how='inner')
merged_df = merged_df[merged_df['repo'] != 'yegor256/jaxec']

In [7]:
result_df = merged_df.copy()
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 605966 entries, 17 to 605999
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   repo       605966 non-null  object 
 1   java_file  605966 non-null  object 
 2   cc         605966 non-null  int64  
 3   nobl       605966 non-null  int64  
 4   loc        605966 non-null  int64  
 5   hse        605966 non-null  float64
 6   hsd        605966 non-null  float64
 7   hsv        605966 non-null  float64
 8   coco       605966 non-null  int64  
 9   midx       605966 non-null  float64
 10  nocl       605966 non-null  int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 55.5+ MB


### Add column bl_ratio = nobl / loc

In [8]:
result_df.loc[:, "bl_ratio"] = merged_df["nobl"] / merged_df["loc"]
result_df.loc[:, "cl_ratio"] = merged_df["nocl"] / merged_df["loc"]

In [9]:
result_df[(result_df['loc'] > 3) & (result_df['loc'] < 40)].head()

,repo,java_file,cc,nobl,loc,hse,hsd,hsv,coco,midx,nocl,bl_ratio,cl_ratio
18,databricks/learning-spark,/src/main/java/com/oreilly/learningsparkexampl...,2,0,8,3621.689,11.500,314.929,1,100.000,0,0.000000,0.000000
19,databricks/learning-spark,/src/main/java/com/oreilly/learningsparkexampl...,1,0,13,9038.089,12.833,704.267,0,100.000,4,0.000000,0.307692
20,databricks/learning-spark,/src/main/java/com/oreilly/learningsparkexampl...,1,0,14,9428.815,13.707,687.888,0,100.000,7,0.000000,0.500000
23,databricks/learning-spark,/src/main/java/com/oreilly/learningsparkexampl...,2,1,12,4849.342,11.974,405.000,1,100.000,0,0.083333,0.000000
24,databricks/learning-spark,/src/main/java/com/oreilly/learningsparkexampl...,2,1,32,32819.043,20.798,1578.000,2,99.414,0,0.031250,0.000000


#### Shape of datasets: all methods, methods with 3 < loc <= 40 and methods with 3 < loc <= 40 and nobl > 0

In [10]:
# Extract the relevant columns
selected_columns = ['cc', 'nobl', 'loc', 'hse', 'hsd', 'hsv', 'coco', 'midx', 'nocl', 'bl_ratio', 'cl_ratio']
subset_df = result_df[selected_columns]
subset_df_3_40 = subset_df[(subset_df['loc'] <= 40) & (subset_df['loc'] > 3)]
subset_df_3_40_with_non_zero_bl = subset_df[(subset_df['loc'] <= 40) & (subset_df['loc'] > 3) & (subset_df['nobl'] > 0)]
subset_df.shape, subset_df_3_40.shape, subset_df_3_40_with_non_zero_bl.shape

((605966, 11), (256559, 11), (78229, 11))

#### Look at total correlation

In [11]:
corr_df1 = subset_df.corr()[['nobl', 'bl_ratio']]
# corr_df1

In [12]:
corr_df2 = subset_df_3_40.corr()[['nobl', 'bl_ratio']]
# corr_df2

In [13]:
corr_df3 = subset_df_3_40_with_non_zero_bl.corr()[['nobl', 'bl_ratio']]
# corr_df3

In [14]:
# Concatenate the DataFrames with a multi-index
keys = ['All methods', 'Methods 3 < loc < 41', 'Methods 3 < loc < 41, nobl > 0']
result_df = pd.concat([corr_df1, corr_df2, corr_df3], axis=1, keys=keys)
result_df

All methods           Methods 3 < loc < 41            \
                nobl  bl_ratio                 nobl  bl_ratio   
cc          0.356849  0.142551             0.274359  0.101978   
nobl        1.000000  0.531144             1.000000  0.823800   
loc         0.726390  0.227269             0.551058  0.275265   
hse         0.325400  0.035472             0.446565  0.220579   
hsd         0.523015  0.276023             0.342076  0.182889   
hsv         0.584066  0.156707             0.535535  0.292651   
coco        0.301803  0.100712             0.287621  0.101423   
midx       -0.583841 -0.127105            -0.208192 -0.073215   
nocl        0.568640  0.251919             0.410333  0.293370   
bl_ratio    0.531144  1.000000             0.823800  1.000000   
cl_ratio    0.145705  0.218926             0.268792  0.281616   

         Methods 3 < loc < 41, nobl > 0            
                                   nobl  bl_ratio  
cc                             0.228543 -0.210173  
nobl                           1.000000  0.599016  
loc                            0.509450 -0.237877  
hse                            0.391224 -0.095871  
hsd                            0.311213 -0.122074  
hsv                            0.481655 -0.130333  
coco                           0.212888 -0.227866  
midx                          -0.213674  0.021376  
nocl                           0.400286  0.221354  
bl_ratio                       0.599016  1.000000  
cl_ratio                       0.256303  0.296787

### Results

- nobl явно коррелирует с loc, даже в методах, где требуется обязательно хотя бы 1 пустая строка (коэфф > 0.5)
- nobl сильно коррелирует с hs* метриками за счет того, что метода становятся не витиеватыми, а перенасыщенными вызывами и внутренней работой (коэфф около 0.3)
- нужно проверить с чем коррелирует nobl, **невелировав** влияние loc, потому что это известная зависимость

#### Look at partial correlation, controlling loc parametr

In [ ]:
control = 'loc'
features = ['nobl', 'bl_ratio', 'cl_ratio']
metrics = ['cc', 'coco', 'hse', 'hsv', 'hsd', 'midx']

partial_correlations = []
# Iterate through feature and metric combinations
for feature in features:
    for metric in metrics:
        # Calculate partial correlation
        partial_corr = pg.partial_corr(data=subset_df, x=metric, y=feature, x_covar=control)
        partial_corr['feature'] = feature
        partial_corr['metric'] = metric
        partial_correlations.append(partial_corr)

print("Correlation between method's property and metric of the code complexity, controlling loc value. All methods")
result_partial_corr = pd.concat(partial_correlations)
result_partial_corr.set_index(['feature', 'metric'], inplace=True)
result_partial_corr

Correlation between method's property and metric of the code complexity, controlling loc value. All methods


n         r           CI95%          p-val
feature  metric                                                 
nobl     cc      605966 -0.217338  [-0.22, -0.21]   0.000000e+00
         coco    605966 -0.099415    [-0.1, -0.1]   0.000000e+00
         hse     605966 -0.073582  [-0.08, -0.07]   0.000000e+00
         hsv     605966 -0.063508  [-0.07, -0.06]   0.000000e+00
         hsd     605966  0.053390    [0.05, 0.06]   0.000000e+00
         midx    605966 -0.006933   [-0.01, -0.0]   6.791614e-08
bl_ratio cc      605966 -0.024500  [-0.03, -0.02]   4.132602e-81
         coco    605966 -0.023685  [-0.03, -0.02]   6.285571e-76
         hse     605966 -0.101462    [-0.1, -0.1]   0.000000e+00
         hsv     605966 -0.069305  [-0.07, -0.07]   0.000000e+00
         hsd     605966  0.167209    [0.16, 0.17]   0.000000e+00
         midx    605966  0.090031    [0.09, 0.09]   0.000000e+00
cl_ratio cc      605966 -0.001415     [-0.0, 0.0]   2.706716e-01
         coco    605966  0.003175     [0.0, 0.01]   1.345193e-02
         hse     605966 -0.032624  [-0.04, -0.03]  2.354636e-142
         hsv     605966 -0.027393  [-0.03, -0.02]  6.322399e-101
         hsd     605966  0.026913    [0.02, 0.03]   1.730218e-97
         midx    605966 -0.001160     [-0.0, 0.0]   3.664764e-01

In [ ]:
partial_correlations_3_40 = []
# Iterate through feature and metric combinations
for feature in features:
    for metric in metrics:
        # Calculate partial correlation
        partial_corr = pg.partial_corr(data=subset_df_3_40, x=metric, y=feature, x_covar=control)
        partial_corr['feature'] = feature
        partial_corr['metric'] = metric
        partial_correlations_3_40.append(partial_corr)

print("Correlation between method's property and metric of the code complexity, controlling loc value. Methods 3 < loc < 41")
result_partial_corr_3_40 = pd.concat(partial_correlations_3_40)
result_partial_corr_3_40.set_index(['feature', 'metric'], inplace=True)
result_partial_corr_3_40

Correlation between method's property and metric of the code complexity, controlling loc value. Methods 3 < loc < 41


n         r           CI95%          p-val
feature  metric                                                 
nobl     cc      257180 -0.146695  [-0.15, -0.14]   0.000000e+00
         coco    257180 -0.124691  [-0.13, -0.12]   0.000000e+00
         hse     257180  0.071403    [0.07, 0.08]  7.838490e-288
         hsv     257180  0.122558    [0.12, 0.13]   0.000000e+00
         hsd     257180  0.007729     [0.0, 0.01]   8.868249e-05
         midx    257180 -0.032262  [-0.04, -0.03]   3.390694e-60
bl_ratio cc      257180 -0.121449  [-0.13, -0.12]   0.000000e+00
         coco    257180 -0.120159  [-0.12, -0.12]   0.000000e+00
         hse     257180  0.030684    [0.03, 0.03]   1.269233e-54
         hsv     257180  0.108839    [0.11, 0.11]   0.000000e+00
         hsd     257180  0.019568    [0.02, 0.02]   3.263261e-23
         midx    257180  0.020248    [0.02, 0.02]   9.665343e-25
cl_ratio cc      257180 -0.039964  [-0.04, -0.04]   2.141701e-91
         coco    257180 -0.021395  [-0.03, -0.02]   1.962977e-27
         hse     257180 -0.007417   [-0.01, -0.0]   1.688061e-04
         hsv     257180  0.001978    [-0.0, 0.01]   3.158209e-01
         hsd     257180 -0.019836  [-0.02, -0.02]   8.265513e-24
         midx    257180 -0.036785  [-0.04, -0.03]   1.030858e-77

In [ ]:
partial_correlations_3_40_with_non_zero_bl = []
# Iterate through feature and metric combinations
for feature in features:
    for metric in metrics:
        # Calculate partial correlation
        partial_corr = pg.partial_corr(data=subset_df_3_40_with_non_zero_bl, x=metric, y=feature, x_covar=control)
        partial_corr['feature'] = feature
        partial_corr['metric'] = metric
        partial_correlations_3_40_with_non_zero_bl.append(partial_corr)

print("Correlation between method's property and metric of the code complexity, controlling loc value. Methods 3 < loc < 41, nobl > 0")
result_partial_corr_3_40_with_non_zero_bl = pd.concat(partial_correlations_3_40_with_non_zero_bl)
result_partial_corr_3_40_with_non_zero_bl.set_index(['feature', 'metric'], inplace=True)
result_partial_corr_3_40_with_non_zero_bl

Correlation between method's property and metric of the code complexity, controlling loc value. Methods 3 < loc < 41, nobl > 0


n         r           CI95%          p-val
feature  metric                                                
nobl     cc      78702 -0.097983   [-0.1, -0.09]  3.980720e-167
         coco    78702 -0.132625  [-0.14, -0.13]  1.166781e-305
         hse     78702  0.073961    [0.07, 0.08]   7.009724e-96
         hsv     78702  0.114605    [0.11, 0.12]  2.835599e-228
         hsd     78702  0.022706    [0.02, 0.03]   1.883299e-10
         midx    78702 -0.045294  [-0.05, -0.04]   5.008688e-37
bl_ratio cc      78702 -0.083803  [-0.09, -0.08]  1.231640e-122
         coco    78702 -0.101322  [-0.11, -0.09]  1.255025e-178
         hse     78702  0.083438    [0.08, 0.09]  1.387996e-121
         hsv     78702  0.113843    [0.11, 0.12]  2.932987e-225
         hsd     78702  0.017296    [0.01, 0.02]   1.218589e-06
         midx    78702 -0.066271  [-0.07, -0.06]   2.583075e-77
cl_ratio cc      78702 -0.043897  [-0.05, -0.04]   7.038890e-35
         coco    78702 -0.033063  [-0.04, -0.03]   1.728968e-20
         hse     78702 -0.009655   [-0.02, -0.0]   6.759197e-03
         hsv     78702 -0.016661  [-0.02, -0.01]   2.951075e-06
         hsd     78702 -0.032321  [-0.04, -0.03]   1.194927e-19
         midx    78702 -0.060012  [-0.07, -0.05]   1.042183e-63

In [ ]:
# Concatenate the DataFrames with a multi-index
keys = ['All methods', 'Methods 3 < loc < 41', 'Methods 3 < loc < 41, nobl > 0']
result_partial_corr_df = pd.concat([result_partial_corr.loc[:, 'r'], result_partial_corr_3_40.loc[:, 'r'], result_partial_corr_3_40_with_non_zero_bl.loc[:, 'r']], axis=1, keys=keys)
result_partial_corr_df

All methods  Methods 3 < loc < 41  \
feature  metric                                      
nobl     cc        -0.217338             -0.146695   
         coco      -0.099415             -0.124691   
         hse       -0.073582              0.071403   
         hsv       -0.063508              0.122558   
         hsd        0.053390              0.007729   
         midx      -0.006933             -0.032262   
bl_ratio cc        -0.024500             -0.121449   
         coco      -0.023685             -0.120159   
         hse       -0.101462              0.030684   
         hsv       -0.069305              0.108839   
         hsd        0.167209              0.019568   
         midx       0.090031              0.020248   
cl_ratio cc        -0.001415             -0.039964   
         coco       0.003175             -0.021395   
         hse       -0.032624             -0.007417   
         hsv       -0.027393              0.001978   
         hsd        0.026913             -0.019836   
         midx      -0.001160             -0.036785   

                 Methods 3 < loc < 41, nobl > 0  
feature  metric                                  
nobl     cc                           -0.097983  
         coco                         -0.132625  
         hse                           0.073961  
         hsv                           0.114605  
         hsd                           0.022706  
         midx                         -0.045294  
bl_ratio cc                           -0.083803  
         coco                         -0.101322  
         hse                           0.083438  
         hsv                           0.113843  
         hsd                           0.017296  
         midx                         -0.066271  
cl_ratio cc                           -0.043897  
         coco                         -0.033063  
         hse                          -0.009655  
         hsv                          -0.016661  
         hsd                          -0.032321  
         midx                         -0.060012

### Results

- коэфф корелляции nobl и hsv = 0.12 в Methods 3 < loc < 41	и Methods 3 < loc < 41, nobl > 0
- коэфф корелляции bl_ratio и hsv = 0.12 в Methods 3 < loc < 41	и Methods 3 < loc < 41, nobl > 0
- отрицательная корреляция nobl/bl_ratio с cc обясняется тем, что условные операторы/switch не разбиваются пробелами
- коэфф корелляции bl_ratio и hsd = 0.16 в All methods